In [5]:
import re
from nltk.tokenize import sent_tokenize

In [6]:
import csv
import string

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Auxiliar functions
- remove_scenes_actions
- extract_trigrams

In [9]:
def remove_scenes_actions(line):
    new = ""
    found = False
    for letter in line:
        if letter == "(" or letter =="[":
            found = True
        if letter == ")" or letter =="]":
            found = False

        #outside brackets
        if found == False and letter!=")" and letter!="]":
            new = new + letter

    
    return(new)

In [10]:
def extract_trigrams(sentence):
    # Step 1
    words_list = sentence.split(" ")

    # Step 2
    trigrams_list = []

    # Step 3
    for i in range(0 , len(words_list) - 2):
        if words_list[i]!='' and words_list[i+1]!='' and words_list[i+2]!='':
            trigram = [words_list[i],words_list[i+1],words_list[i + 2]]
            trigrams_list.append(trigram)
    return trigrams_list

In [ ]:
file = open("friends.csv", 'r')
csvreader = csv.reader(file)
data = list(csvreader)

In [ ]:
characters =["Rachel", "Monica", "Joey", "Chandler", "Ross", "Phoebe"]

In [ ]:
stop_words = set(stopwords.words('english'))

*************** UNIGRAMS ***********************************************

In [ ]:
for character in characters:
        lines = []
        sp_character = []
        for row in data:
            if row[-2] == character:
                #we only want dialogue
                str = remove_scenes_actions(row[-1])
                lines.append(str)

In [ ]:
        #create auxiliar vector WITHOUT punctuation for TF-IDF purposes
        words = []
        for line in lines:
            #remve punctuation
            line = line.translate(str.maketrans('', '', string.punctuation))
            sp_character.append(line)
            line = word_tokenize(line)
            for word in line:
                words.append(word.lower())

In [ ]:
        
        counts = Counter(words)
        counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        print(character)
        if (character=="Monica"):
            for elem in counts:
                if elem[0] not in stop_words:
                    print(elem)
        print("***************************************")